In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.callbacks as cb
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from functools import partial
from project import *

In [2]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
MAX_LENGTH = 200

In [3]:
train_examples, test_examples, dataset_info = get_datasets()
encoder = dataset_info.features['text'].encoder
train_dataset, test_dataset = transform_datasets(train_examples, test_examples, 
                                                 encoder=encoder, 
                                                 batch_size=BATCH_SIZE,  
                                                 max_length=MAX_LENGTH, 
                                                 buffer_size=BUFFER_SIZE)

In [4]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

input_vocab_size = encoder.vocab_size
dropout_rate = 0.1

In [5]:
loss_function = tf.keras.losses.BinaryCrossentropy()
transformer = TransformerEncoderClassifier(num_layers, d_model, num_heads, dff, 
                                           input_vocab_size, 
                                           pe_input=input_vocab_size, 
                                           rate=dropout_rate)
transformer.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

checkpoint = tf.train.latest_checkpoint("./checkpoints")
if checkpoint is not None:
    print("Loading previously trained model.")
    transformer.load_weights(checkpoint)

In [6]:
EPOCHS = 100

In [ ]:
tb = cb.TensorBoard(histogram_freq=1, embeddings_freq=1)
csv = cb.CSVLogger('training.csv')
early = cb.EarlyStopping(monitor='val_accuracy', patience=5)
save = cb.ModelCheckpoint(filepath="checkpoints/train",
         monitor='val_accuracy',
         save_best_only=True,
         save_weights_only=True)

model_history = transformer.fit(train_dataset, 
                                validation_data=test_dataset,  
                                validation_freq=1,
                                shuffle=False,
                                callbacks=[early, save, tb, csv],
                                epochs=EPOCHS)

Epoch 1/100
134/134 [==============================]8441 - accuracy: 0.484 - 4s 2s/step - loss: 265.6336 - accuracy: 0.51 - 4s 1s/step - loss: 328.0171 - accuracy: 0.51 - 4s 1s/step - loss: 308.1908 - accuracy: 0.50 - 4s 880ms/step - loss: 253.1093 - accuracy: 0.478 - 4s 748ms/step - loss: 217.0679 - accuracy: 0.474 - 5s 653ms/step - loss: 189.2216 - accuracy: 0.491 - 5s 583ms/step - loss: 168.7211 - accuracy: 0.496 - 5s 528ms/step - loss: 150.3873 - accuracy: 0.493 - 5s 484ms/step - loss: 135.5370 - accuracy: 0.485 - 5s 448ms/step - loss: 123.3939 - accuracy: 0.487 - 5s 418ms/step - loss: 113.3892 - accuracy: 0.479 - 5s 392ms/step - loss: 105.9875 - accuracy: 0.472 - 5s 371ms/step - loss: 98.9813 - accuracy: 0.471 - 5s 352ms/step - loss: 93.8005 - accuracy: 0.47 - 5s 336ms/step - loss: 89.4048 - accuracy: 0.48 - 5s 321ms/step - loss: 85.4680 - accuracy: 0.47 - 6s 308ms/step - loss: 81.1113 - accuracy: 0.48 - 6s 297ms/step - loss: 77.9230 - accuracy: 0.47 - 6s 286ms/step - loss: 74.423

134/134 [==============================] - ETA: 17s - loss: 9.8014 - accuracy: 0.453 - ETA: 14s - loss: 7.3043 - accuracy: 0.523 - ETA: 13s - loss: 5.6236 - accuracy: 0.500 - ETA: 13s - loss: 4.9957 - accuracy: 0.527 - ETA: 12s - loss: 4.3336 - accuracy: 0.525 - ETA: 12s - loss: 4.0980 - accuracy: 0.533 - ETA: 12s - loss: 3.6924 - accuracy: 0.537 - ETA: 12s - loss: 3.5368 - accuracy: 0.535 - ETA: 11s - loss: 3.3898 - accuracy: 0.531 - ETA: 11s - loss: 3.2073 - accuracy: 0.534 - ETA: 11s - loss: 3.2171 - accuracy: 0.535 - ETA: 11s - loss: 3.1601 - accuracy: 0.533 - ETA: 11s - loss: 3.3407 - accuracy: 0.525 - ETA: 11s - loss: 3.2434 - accuracy: 0.523 - ETA: 11s - loss: 3.3542 - accuracy: 0.527 - ETA: 11s - loss: 3.5767 - accuracy: 0.522 - ETA: 10s - loss: 3.4316 - accuracy: 0.524 - ETA: 10s - loss: 3.5590 - accuracy: 0.524 - ETA: 10s - loss: 3.5561 - accuracy: 0.528 - ETA: 10s - loss: 3.5422 - accuracy: 0.523 - ETA: 10s - loss: 3.4863 - accuracy: 0.523 - ETA: 10s - loss: 3.5157 - accurac

134/134 [==============================] - ETA: 18s - loss: 1.4519 - accuracy: 0.531 - ETA: 15s - loss: 3.0767 - accuracy: 0.500 - ETA: 14s - loss: 2.9975 - accuracy: 0.500 - ETA: 13s - loss: 2.8873 - accuracy: 0.519 - ETA: 13s - loss: 3.1228 - accuracy: 0.525 - ETA: 12s - loss: 2.8037 - accuracy: 0.510 - ETA: 12s - loss: 2.8022 - accuracy: 0.495 - ETA: 12s - loss: 2.5803 - accuracy: 0.507 - ETA: 12s - loss: 2.5428 - accuracy: 0.506 - ETA: 11s - loss: 2.3904 - accuracy: 0.510 - ETA: 11s - loss: 2.5246 - accuracy: 0.494 - ETA: 11s - loss: 2.4678 - accuracy: 0.492 - ETA: 11s - loss: 2.3732 - accuracy: 0.494 - ETA: 11s - loss: 2.3000 - accuracy: 0.496 - ETA: 11s - loss: 2.2247 - accuracy: 0.497 - ETA: 11s - loss: 2.1910 - accuracy: 0.493 - ETA: 10s - loss: 2.1396 - accuracy: 0.496 - ETA: 10s - loss: 2.0820 - accuracy: 0.493 - ETA: 10s - loss: 2.0934 - accuracy: 0.494 - ETA: 10s - loss: 2.0468 - accuracy: 0.496 - ETA: 10s - loss: 2.0939 - accuracy: 0.497 - ETA: 10s - loss: 2.0568 - accurac

134/134 [==============================] - ETA: 19s - loss: 2.6614 - accuracy: 0.468 - ETA: 15s - loss: 2.2346 - accuracy: 0.523 - ETA: 14s - loss: 2.0910 - accuracy: 0.520 - ETA: 13s - loss: 1.9007 - accuracy: 0.531 - ETA: 13s - loss: 1.8224 - accuracy: 0.546 - ETA: 12s - loss: 1.7725 - accuracy: 0.536 - ETA: 12s - loss: 1.8174 - accuracy: 0.529 - ETA: 12s - loss: 1.8005 - accuracy: 0.525 - ETA: 12s - loss: 1.9436 - accuracy: 0.526 - ETA: 11s - loss: 1.9921 - accuracy: 0.539 - ETA: 11s - loss: 1.8954 - accuracy: 0.536 - ETA: 11s - loss: 1.9629 - accuracy: 0.526 - ETA: 11s - loss: 1.8957 - accuracy: 0.530 - ETA: 11s - loss: 1.8541 - accuracy: 0.534 - ETA: 11s - loss: 1.7967 - accuracy: 0.538 - ETA: 11s - loss: 1.7519 - accuracy: 0.539 - ETA: 10s - loss: 1.7010 - accuracy: 0.539 - ETA: 10s - loss: 1.6750 - accuracy: 0.535 - ETA: 10s - loss: 1.6281 - accuracy: 0.538 - ETA: 10s - loss: 1.5974 - accuracy: 0.540 - ETA: 10s - loss: 1.5765 - accuracy: 0.542 - ETA: 10s - loss: 1.5602 - accurac

134/134 [==============================] - ETA: 17s - loss: 0.5169 - accuracy: 0.859 - ETA: 14s - loss: 0.4549 - accuracy: 0.851 - ETA: 13s - loss: 0.7139 - accuracy: 0.843 - ETA: 13s - loss: 0.7798 - accuracy: 0.824 - ETA: 12s - loss: 0.7222 - accuracy: 0.831 - ETA: 12s - loss: 0.6503 - accuracy: 0.838 - ETA: 12s - loss: 0.6186 - accuracy: 0.846 - ETA: 12s - loss: 0.6386 - accuracy: 0.845 - ETA: 11s - loss: 0.6507 - accuracy: 0.840 - ETA: 11s - loss: 0.6523 - accuracy: 0.839 - ETA: 11s - loss: 0.6489 - accuracy: 0.835 - ETA: 11s - loss: 0.6302 - accuracy: 0.833 - ETA: 11s - loss: 0.6199 - accuracy: 0.834 - ETA: 11s - loss: 0.6173 - accuracy: 0.831 - ETA: 11s - loss: 0.6011 - accuracy: 0.833 - ETA: 11s - loss: 0.6126 - accuracy: 0.827 - ETA: 10s - loss: 0.6031 - accuracy: 0.829 - ETA: 10s - loss: 0.6068 - accuracy: 0.827 - ETA: 10s - loss: 0.6039 - accuracy: 0.828 - ETA: 10s - loss: 0.6020 - accuracy: 0.825 - ETA: 10s - loss: 0.6049 - accuracy: 0.820 - ETA: 10s - loss: 0.6007 - accurac

133/137 [============================>.] - ETA: 17s - loss: 0.7632 - accuracy: 0.812 - ETA: 14s - loss: 0.6057 - accuracy: 0.812 - ETA: 13s - loss: 0.4876 - accuracy: 0.854 - ETA: 12s - loss: 0.6131 - accuracy: 0.812 - ETA: 12s - loss: 0.5336 - accuracy: 0.837 - ETA: 12s - loss: 0.5347 - accuracy: 0.841 - ETA: 12s - loss: 0.4925 - accuracy: 0.850 - ETA: 11s - loss: 0.4488 - accuracy: 0.861 - ETA: 11s - loss: 0.4472 - accuracy: 0.864 - ETA: 11s - loss: 0.4361 - accuracy: 0.871 - ETA: 11s - loss: 0.4147 - accuracy: 0.877 - ETA: 11s - loss: 0.3844 - accuracy: 0.886 - ETA: 11s - loss: 0.3654 - accuracy: 0.890 - ETA: 11s - loss: 0.3534 - accuracy: 0.894 - ETA: 11s - loss: 0.3454 - accuracy: 0.895 - ETA: 10s - loss: 0.3319 - accuracy: 0.898 - ETA: 10s - loss: 0.3300 - accuracy: 0.900 - ETA: 10s - loss: 0.3379 - accuracy: 0.901 - ETA: 10s - loss: 0.3278 - accuracy: 0.902 - ETA: 10s - loss: 0.3172 - accuracy: 0.905 - ETA: 10s - loss: 0.3271 - accuracy: 0.904 - ETA: 10s - loss: 0.3182 - accurac

In [ ]:
model_history.history

In [ ]:
for key in model_history.history:
    print(key)

## Example of Evaluating a model

In [ ]:
eval_loss, eval_acc = transformer.evaluate(test_dataset)

In [ ]:
print(f'Loss: {eval_loss}, Acc: {eval_acc}')

### Examples of sentiment analysis

In [ ]:
sent = partial(sentiment, encoder=encoder, transformer=transformer)

In [ ]:
sent("Terrible movie")

In [ ]:
sent('We loved the movie and definitely recommend it!')